In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Data Visualization and Exploratory Data Analysis Lab
## Visualizing and exploring data. The data mining process

In this lab, you'll get acquainted with the most streamed songs on Spotify in 2024. The dataset and its associated metadata can be found [here](https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024). The version you'll need is provided in the `data/` folder.

You know the drill. Do what you can / want / need to answer the questions to the best of your ability. Answers do not need to be trivial, or even the same among different people.

### Problem 1. Read the dataset (1 point)
Read the file without unzipping it first. You can try a different character encoding, like `unicode_escape`. Don't worry too much about weird characters.

In [7]:


# Read the dataset from zip file or CSV, with error handling
zip_path = 'data/spotify_most_streamed_2024.zip'
csv_path = 'data/spotify_most_streamed_2024.csv'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        csv_file = zip_ref.namelist()[0]  # Get the first (and likely only) CSV file
        with zip_ref.open(csv_file) as file:
            df = pd.read_csv(file, encoding='unicode_escape')
elif os.path.exists(csv_path):
    try:
        df = pd.read_csv(csv_path, encoding='unicode_escape')
    except Exception:
        df = pd.read_csv(csv_path, encoding='utf-8')
else:
    print("Warning: Neither the zip file nor the CSV file was found in the 'data/' directory.")
    df = None  # Set df to None so subsequent code can check if data was loaded

print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()

Dataset shape: (4600, 29)

Column names:
['Track', 'Album Name', 'Artist', 'Release Date', 'ISRC', 'All Time Rank', 'Track Score', 'Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach', 'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach', 'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins', 'Deezer Playlist Count', 'Deezer Playlist Reach', 'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations', 'Soundcloud Streams', 'Shazam Counts', 'TIDAL Popularity', 'Explicit Track']

First few rows:


,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


### Problem 2. Perform some cleaning (1 point)
Ensure all data has been read correctly; check the data types. Give the columns better names (e.g. `all_time_rank`, `track_score`, etc.). To do so, try to use `apply()` instead of a manual mapping between old and new name. Get rid of any unnecessary ones.

In [8]:

# Ensure all data has been read correctly; check the data types. 
# Give the columns better names (e.g. `all_time_rank`, `track_score`, etc.). 
# To do so, try to use `apply()` instead of a manual mapping between old and new name. 
# Get rid of any unnecessary ones.

# Step 1: Check current state of the data
print("=== STEP 1: Current Data Overview ===")
print(f"Dataset shape: {df.shape}")
print(f"Number of columns: {len(df.columns)}")
print("\nCurrent column names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# Step 2: Check data types
print("\n=== STEP 2: Data Types ===")
print(df.dtypes)

# Step 3: Get detailed info about the dataset
print("\n=== STEP 3: Dataset Info ===")
df.info()

# Step 4: Check for missing values
print("\n=== STEP 4: Missing Values ===")
missing_values = df.isnull().sum()
print("Columns with missing values:")
for col, missing_count in missing_values[missing_values > 0].items():
    percentage = (missing_count / len(df)) * 100
    print(f"{col}: {missing_count} ({percentage:.1f}%)")

# Step 5: Look at first few rows to understand the data better
print("\n=== STEP 5: Sample Data ===")
print("First 3 rows:")
print(df.head(3))

# Step 6: Define function to clean column names using apply()
def clean_column_name(col_name):
    """
    Function to clean and standardize column names
    Converts to snake_case and makes names more descriptive
    """
    # Convert to lowercase and replace spaces/special characters
    cleaned = col_name.lower()
    cleaned = cleaned.replace(' ', '_')
    cleaned = cleaned.replace('-', '_')
    cleaned = cleaned.replace('(', '').replace(')', '')
    cleaned = cleaned.replace('%', 'percent')
    cleaned = cleaned.replace('#', 'number')
    
    # Handle specific column name improvements
    if 'all_time_rank' in cleaned or 'all time rank' in cleaned:
        return 'all_time_rank'
    elif 'track_score' in cleaned or 'track score' in cleaned:
        return 'track_score'
    elif cleaned == 'track':
        return 'track_name'
    elif cleaned == 'artist':
        return 'artist_name'
    elif 'spotify_streams' in cleaned:
        return 'spotify_streams'
    elif 'spotify_playlist_count' in cleaned or 'spotify playlists' in cleaned:
        return 'spotify_playlist_count'
    elif 'spotify_playlist_reach' in cleaned:
        return 'spotify_playlist_reach'
    elif 'youtube_views' in cleaned:
        return 'youtube_views'
    elif 'youtube_likes' in cleaned:
        return 'youtube_likes'
    elif 'tiktok_posts' in cleaned:
        return 'tiktok_posts'
    elif 'tiktok_likes' in cleaned:
        return 'tiktok_likes'
    elif 'tiktok_views' in cleaned:
        return 'tiktok_views'
    elif 'apple_playlist' in cleaned:
        if 'count' in cleaned:
            return 'apple_playlist_count'
        elif 'reach' in cleaned:
            return 'apple_playlist_reach'
    elif 'deezer_playlist' in cleaned:
        if 'count' in cleaned:
            return 'deezer_playlist_count'
        elif 'reach' in cleaned:
            return 'deezer_playlist_reach'
    elif 'amazon_playlist' in cleaned:
        if 'count' in cleaned:
            return 'amazon_playlist_count'
        elif 'reach' in cleaned:
            return 'amazon_playlist_reach'
    elif 'pandora' in cleaned:
        if 'streams' in cleaned:
            return 'pandora_streams'
        elif 'track_stations' in cleaned:
            return 'pandora_track_stations'
    elif 'soundcloud_streams' in cleaned:
        return 'soundcloud_streams'
    elif 'shazam_counts' in cleaned:
        return 'shazam_counts'
    elif 'explicit' in cleaned:
        return 'is_explicit'
    elif 'release_date' in cleaned:
        return 'release_date'
    
    # If no specific mapping, return the cleaned version
    return cleaned

# Step 7: Apply the cleaning function to all column names
print("\n=== STEP 7: Applying Column Name Cleaning ===")
print("Before cleaning:")
print(list(df.columns))

# Use apply() to clean all column names
df.columns = df.columns.to_series().apply(clean_column_name)

print("\nAfter cleaning:")
print(list(df.columns))

# Step 8: Convert data types appropriately
print("\n=== STEP 8: Converting Data Types ===")

# Identify numeric columns that might be stored as strings
potential_numeric_columns = [
    'all_time_rank', 'track_score', 'spotify_streams', 'spotify_playlist_count',
    'spotify_playlist_reach', 'youtube_views', 'youtube_likes', 'tiktok_posts',
    'tiktok_likes', 'tiktok_views', 'apple_playlist_count', 'apple_playlist_reach',
    'deezer_playlist_count', 'deezer_playlist_reach', 'amazon_playlist_count',
    'amazon_playlist_reach', 'pandora_streams', 'pandora_track_stations',
    'soundcloud_streams', 'shazam_counts'
]

# Convert columns that exist in our dataframe
for col in potential_numeric_columns:
    if col in df.columns:
        print(f"Converting {col} to numeric...")
        # Remove commas and convert to numeric
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '')
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert boolean columns
if 'is_explicit' in df.columns:
    print("Converting explicit column to boolean...")
    df['is_explicit'] = df['is_explicit'].map({'True': True, 'False': False, True: True, False: False})

# Convert date columns
if 'release_date' in df.columns:
    print("Converting release_date to datetime...")
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    
    # Extract year and month for easier analysis
    df['release_year'] = df['release_date'].dt.year
    df['release_month'] = df['release_date'].dt.month
    df['release_month_name'] = df['release_date'].dt.month_name()

# Step 9: Identify and remove unnecessary columns
print("\n=== STEP 9: Identifying Unnecessary Columns ===")

# Define essential columns for analysis
essential_columns = [
    'track_name', 'artist_name', 'release_date', 'release_year', 'release_month',
    'is_explicit', 'spotify_streams', 'spotify_playlist_count', 'youtube_views',
    'youtube_likes', 'tiktok_posts', 'tiktok_likes', 'tiktok_views',
    'apple_playlist_count', 'deezer_playlist_count', 'amazon_playlist_count',
    'all_time_rank', 'track_score'
]

# Keep only columns that exist in our dataframe
existing_essential_columns = [col for col in essential_columns if col in df.columns]
other_columns = [col for col in df.columns if col not in essential_columns]

print("Essential columns found:", existing_essential_columns)
print("Other columns:", other_columns)

# Create cleaned dataframe with only essential columns
df_clean = df[existing_essential_columns].copy()

print(f"\nOriginal dataset: {df.shape}")
print(f"Cleaned dataset: {df_clean.shape}")

# Step 10: Final data overview
print("\n=== STEP 10: Final Cleaned Data Overview ===")
print("Final column names and types:")
for col in df_clean.columns:
    print(f"{col}: {df_clean[col].dtype}")

print(f"\nFinal missing values:")
missing_final = df_clean.isnull().sum()
for col, missing_count in missing_final[missing_final > 0].items():
    percentage = (missing_count / len(df_clean)) * 100
    print(f"{col}: {missing_count} ({percentage:.1f}%)")

print("\nFirst 3 rows of cleaned data:")
print(df_clean.head(3))

print("\n=== CLEANING COMPLETE ===")
print(f"✓ Cleaned {len(df.columns)} columns to {len(df_clean.columns)} essential columns")
print(f"✓ Renamed columns using apply() function for consistency")
print(f"✓ Converted data types appropriately")
print(f"✓ Removed unnecessary columns")
print(f"✓ Added derived columns (release_year, release_month)")

=== STEP 1: Current Data Overview ===
Dataset shape: (4600, 29)
Number of columns: 29

Current column names:
 1. Track
 2. Album Name
 3. Artist
 4. Release Date
 5. ISRC
 6. All Time Rank
 7. Track Score
 8. Spotify Streams
 9. Spotify Playlist Count
10. Spotify Playlist Reach
11. Spotify Popularity
12. YouTube Views
13. YouTube Likes
14. TikTok Posts
15. TikTok Likes
16. TikTok Views
17. YouTube Playlist Reach
18. Apple Music Playlist Count
19. AirPlay Spins
20. SiriusXM Spins
21. Deezer Playlist Count
22. Deezer Playlist Reach
23. Amazon Playlist Count
24. Pandora Streams
25. Pandora Track Stations
26. Soundcloud Streams
27. Shazam Counts
28. TIDAL Popularity
29. Explicit Track

=== STEP 2: Data Types ===
Track                          object
Album Name                     object
Artist                         object
Release Date                   object
ISRC                           object
All Time Rank                  object
Track Score                   float64
Spotify Streams 

### Problem 3. Most productive artists (1 point)
Who are the five artists with the most songs in the dataset?

Who are the five "clean-mouthed" artists (i.e., with no explicit songs)? **Note:** We're not going into details but we can start a discussion about whether a song needs swearing to be popular.

### Problem 4. Most streamed artists (1 point)
And who are the top five most streamed (as measured by Spotify streams) artists?

### Problem 5. Songs by year and month (1 point)
How many songs have been released each year? Present an appropriate plot. Can you explain the behavior of the plot for 2024?

How about months? Is / Are there (a) popular month(s) to release music?

### Problem 6. Playlists (2 points)
Is there any connection (correlation) between users adding a song to playlists in one service, or another? Only Spotify, Apple, Deezer, and Amazon offer the ability to add a song to a playlist. Find a way to plot all these relationships at the same time, and analyze them. Experiment with different types of correlations.

### Problem 7. YouTube views and likes (1 point)
What is the relationship between YouTube views and likes? Present an appropriate plot. 

What is the mean YouTube views-to-likes ratio? What is its distribution? Find a way to plot it and describe it.

### Problem 8. TikTok stuff (2 points)
The most popular songs on TikTok released every year show... interesting behavior. Which years peaked the most TikTok views? Show an appropriate chart. Can you explain this behavior? For a bit of context, TikTok was created in 2016.

Now, how much popular is the most popular song for each release year, than the mean popularity? Analyze the results.

In both parts, it would be helpful to see the actual songs.

### * Problem 9. Explore (and clean) at will
There is a lot to look for here. For example, you can easily link a song to its genres, and lyrics. You may also try to link artists and albums to more info about them. Or you can compare and contrast a song's performance across different platforms, in a similar manner to what you already did above; maybe even assign a better song ranking system (across platforms with different popularity metrics, and different requirements) than the one provided in the dataset.